# Single Rotating Radar Simulation & Single-Target Tracking

This script demonstrates a simulation of a single 2-D (Range-Bearing) Radar, which is mounted on a non-stationary platform and generates measurements of a single (also non-stationary) ground truth target. 

Additionally, an EKF and UKF Predictor/Updater pairs are employed to estimate the state of the ground truth target, given the measurements generated by the radar.

In [ ]:
# Import external dependencies
import datetime
import copy
import numpy as np

# Import from Stone Soup
from stonesoup.types.hypothesis import Hypothesis
from stonesoup.types.state import State, GaussianState
from stonesoup.types.array import StateVector, CovarianceMatrix
from stonesoup.functions import pol2cart
from stonesoup.models.transition.linear import (
    ConstantVelocity,
    CombinedLinearGaussianTransitionModel
)
from stonesoup.platform.simple import SensorPlatform
from stonesoup.sensor.radar import RadarRotatingRangeBearing
from stonesoup.simulator.simple import SingleTargetGroundTruthSimulator
from stonesoup.predictor.kalman import (
    ExtendedKalmanPredictor,
    UnscentedKalmanPredictor
)
from stonesoup.updater.kalman import (
    ExtendedKalmanUpdater,
    UnscentedKalmanUpdater
)

# Plotting
from matplotlib import pyplot as plt

# Create figure
fig = plt.figure()

In [ ]:
def plot_data(groundtruth_paths, detections, tracks_ekf, tracks_ukf,
              platform, radar_position_list, radar_orientation_list,
              radar, frame_index=None, interactive = False):

    plt.clf()
    if(interactive):
        % matplotlib notebook
    plt.rcParams['figure.figsize'] = (8, 8)
    plt.style.use('seaborn-colorblind')

    # Plot GroundTruth tracks
    for path in groundtruth_paths:
        data = np.array([state.state_vector for state in path])
        if(frame_index is None):
            frame_index = data.shape[0]-1
        plt.plot(data[:frame_index+1, 0],
                 data[:frame_index+1, 2], linestyle='-', marker='')

    # Plot the platform position
    data = np.array([state for state in platform])
    plt.plot(data[:frame_index+1, 0], data[:frame_index+1, 2],
             linestyle='-', marker='.')

    # Plot the sensor FOV
    origin = radar_position_list[frame_index]
    orientation = radar_orientation_list[frame_index]
    fov = radar.fov_angle
    fov_min = orientation-fov/2
    fov_max = orientation+fov/2
    [x, y] = pol2cart(100, fov_min)
    plt.quiver(*origin, x, y, color=['r', 'b', 'g'], scale=1, width=0.001)
    [x, y] = pol2cart(100, fov_max)
    plt.quiver(*origin, x, y, color=['r', 'b', 'g'], scale=1, width=0.001)

    # Plot the detections
    data = np.array([state for idx, state in enumerate(
        detections) if (state is not None and idx < frame_index+1)])
    if(data.shape[0] > 0):
        plt.plot(data[:, 0],
                 data[:, 1], linestyle='', marker='2')

    # Plot the estimated trajectory (EKF)
    data = np.array([state for state in tracks_ekf])
    if(data.shape[0] > 0):
        plt.plot(data[:frame_index+1, 0],
                 data[:frame_index+1, 2], linestyle='-', marker='x')
    # Plot the estimated trajectory (UKF)
    data = np.array([state for state in tracks_ukf])
    if(data.shape[0] > 0):
        plt.plot(data[:frame_index+1, 0],
                 data[:frame_index+1, 2], linestyle='-', marker='x')

    plt.axis('equal')
    plt.xlabel("$x$")
    plt.ylabel("$y$")
    plt.legend(['Target Ground Truth', 'Platform/Sensor Trajectory',
                'Detections', 'Estimated Trajectory (EKF)',
                'Estimated Trajectory (UKF)', 'Radar Field Of View'])
    plt.pause(0.01)

Add Target GroundTruth Simulator
--------------------------------------------------

In [ ]:
# Set up target's transition model
target_transition_model = CombinedLinearGaussianTransitionModel(
    (ConstantVelocity(1), ConstantVelocity(1)))

# Simulation start timestamp & Timestep duration
timestamp_init = datetime.datetime.now()
timestep_duration = datetime.timedelta(seconds=0.05)  # 0.1sec

# Number of timesteps to simulate for
num_sim_iter = 500

# Set target ground truth prior
ground_truth_prior_state_vector = StateVector(
    [
        [np.random.rand()*100-50],
        [np.random.rand()*10-5],
        [np.random.rand()*100-50],
        [np.random.rand()*10-5]
    ]
)
ground_truth_prior = GaussianState(
    ground_truth_prior_state_vector,
    CovarianceMatrix(np.diag([10, 1, 10, 1])),
    timestamp=timestamp_init
)

# Generate a single target ground truth simulator
groundtruth_sim = SingleTargetGroundTruthSimulator(
    transition_model=target_transition_model,
    initial_state=ground_truth_prior,
    timestep=timestep_duration,
    number_steps=num_sim_iter
)
# Create ground truth generator function
gts = groundtruth_sim.groundtruth_paths_gen()

Setup a Radar sensor
-------------------------------

In [ ]:
# Generate a radar sensor
noise_covar = CovarianceMatrix(np.array([[0.001, 0],
                                         [0, 1]]))
# Set the radar position and orientation
# (dummy, since they will be overwitten when attached to platform)
radar_position = StateVector(np.array([[0], [0]]))
radar_orientation = StateVector(np.array([[0], [0], [0]]))

# Set the dwell offset of the radar head relative to the orientation
# of the sensors
dwell_center = State(StateVector([[0]]), copy.deepcopy(timestamp_init))

# Instantiate the rotating radar
radar = RadarRotatingRangeBearing(radar_position, 
                                  radar_orientation, 
                                  ndim_state = 4,
                                  mapping = np.array([0, 2]), 
                                  noise_covar = noise_covar, 
                                  dwell_center = dwell_center,
                                  rpm = 20, 
                                  max_range = 100000, 
                                  fov_angle = np.pi/16)

Add a Platform and attach the sensor on it
------------------------------------------------------------

In [ ]:
# Define transition model and position for 2D platform
platform_transition_model = CombinedLinearGaussianTransitionModel(
    [ConstantVelocity(1), ConstantVelocity(1)])

# Initial platform state
platform_prior_state_vector = StateVector(
    [
        [np.random.rand()*100-50],
        [np.random.rand()*10-5],
        [np.random.rand()*100-50],
        [np.random.rand()*10-5]
    ]
)
platform_state = State(platform_prior_state_vector, timestamp_init)

# Define a mounting offset for a sensor relative to the platform
mounting_offsets = np.array([[0, 0]])
# This defines the mapping to the platforms state vector (i.e. x and y)
mounting_mappings = np.array([[0, 2]])
# create a platform with the simple radar mounted
platform = SensorPlatform(platform_state,
                          platform_transition_model,
                          sensors = [radar],
                          mounting_offsets = mounting_offsets,
                          mounting_mappings = mounting_mappings)

# Simulate

In [ ]:
groundtruth_paths = set()  # Store for plotting later
platform_position_list = []  # Store platform path
radar_position_list = []
radar_orientation_list = []
measurement_pol_list = []
measurement_cart_list = []
estimates_list_ekf = []
estimates_list_ukf = []

# Simulate
timestamp = timestamp_init
for i in range(num_sim_iter):

    # next on Ground truth simulator
    next(gts)

    # Generate and extract ground truth state
    # set union useful for fiuture with multi target :)
    groundtruth_paths |= groundtruth_sim.groundtruth_paths
    target_state = groundtruth_sim.groundtruth_paths.pop()[-1]

    # Move platform
    platform.move(timestamp)

    # Store position and oritentation of platform/sensor(s)
    platform_position_list.append(platform.state.state_vector)
    radar_position_list.append(radar.position)

    # Measure the target location
    measurement_pol = platform.sensors[0].gen_measurement(target_state)
    radar_orientation_list.append(
        radar.dwell_center.state_vector[0, 0]+radar.orientation[2, 0])
    measurement_pol_list.append(measurement_pol)
    if measurement_pol is not None:

        # Transform back to global co-ordinate frame for plotting
        rot_mat = measurement_pol.measurement_model._rotation_matrix
        measurement_cart = np.linalg.pinv(rot_mat[0:2, 0:2])\
            @pol2cart(measurement_pol.state_vector[1][0],
                      measurement_pol.state_vector[0][0])

        measurement_cart_list.append(
            StateVector(
                [[measurement_cart[0]
                  + measurement_pol.measurement_model.translation_offset[0][0]],  # noqa:E501
                 [measurement_cart[1]
                  + measurement_pol.measurement_model.translation_offset[1][0]]
                 ]
            )
        )
    else:
        measurement_cart_list.append(None)
    timestamp = timestamp + timestep_duration

# Estimate

In [ ]:
# Estimate
timestamp = timestamp_init

# Assume we know the target's initial state
state_prior_ekf = copy.copy(ground_truth_prior)
state_prior_ukf = copy.copy(ground_truth_prior)

ekf_predictor = ExtendedKalmanPredictor(target_transition_model)
ekf_updater = ExtendedKalmanUpdater(
    measurement_model=radar.measurement_model)
ukf_predictor = UnscentedKalmanPredictor(target_transition_model)
ukf_updater = UnscentedKalmanUpdater(
    measurement_model=radar.measurement_model)

for i in range(num_sim_iter):

    # Extract simulated measurement
    measurement_pol = measurement_pol_list[i]

    # Predict the target position using EKF
    state_prediction_ekf = ekf_predictor.predict(
        state_prior_ekf, timestamp=timestamp)

    # Predict the target position using UKF
    state_prediction_ukf = ukf_predictor.predict(
        state_prior_ukf, timestamp=timestamp)

    # If a measurement of the target is received
    if measurement_pol is not None:
        # Update/Correct the target position using EKF
        hypothesis = Hypothesis(state_prediction_ekf, measurement_pol)
        state_prior_ekf = ekf_updater.update(hypothesis=hypothesis)
        estimates_list_ekf.append(state_prior_ekf.state_vector)

        # Update/Correct the target position using UKF
        hypothesis = Hypothesis(state_prediction_ukf, measurement_pol)
        state_prior_ukf = ukf_updater.update(hypothesis=hypothesis)
        estimates_list_ukf.append(state_prior_ukf.state_vector)
    else:
        # Store the 
        state_prior_ekf = state_prediction_ekf
        estimates_list_ekf.append(state_prior_ekf.state_vector)

        # Estimate the target position using UKF
        state_prior_ukf = state_prediction_ukf
        estimates_list_ukf.append(state_prior_ukf.state_vector)
    
    # Uncomment code below to plot all frames 
    # (WARNING: It takes a while and eats up RAM space!)
    plot_data(groundtruth_paths, measurement_cart_list, estimates_list_ekf,
              estimates_list_ukf, platform_position_list, radar_position_list,
              radar_orientation_list, radar, i)

    timestamp += timestep_duration

# Plot output

In [ ]:
plt.clf()
plot_data(groundtruth_paths, measurement_cart_list, estimates_list_ekf,
          estimates_list_ukf, platform_position_list, radar_position_list,
          radar_orientation_list, radar, i, True)
plt.show()